# CZI WSI Inference
Adapt the WSI inference code from YOLO Braak Stage project to utilize the czifile Python library to tile the WSI instead of large image. The motivation for this is because large image library is not compatible with CZI files that UC Davis primarily uses.

In [23]:
# Imports
import czifile
from time import perf_counter
from tqdm.notebook import tqdm
from pathlib import Path
import cv2 as cv
import pandas as pd
from multiprocessing import Pool
from colorama import Fore, Style
from os.path import isfile
from shapely.geometry import Polygon
from geopandas import GeoDataFrame

from nft_helpers.yolov5 import predict
from nft_helpers.yolov5.utils import read_yolo_label, remove_contained_boxes, non_max_suppression
from nft_helpers.utils import im_to_txt_path

In [24]:
# Constants
CZI_FILEPATH = "/workspace/data/images/NA5090-02_AT8.czi"
SCAN_MAG = 89.93645454545455 
TILE_MAG = 40
TILE_SIZE = 1280
STRIDE = 960
SAVED_DIR = "/workspace/data/tiles"
RGB_FILL = (114, 114, 114)
NPROC = 20
NMS_IOU_THR = 0.6
CONTAINED_THR = 0.8

In [6]:
# Read the CZI file using the czifile Python library.
t_start = perf_counter()
czi_img = czifile.imread(CZI_FILEPATH)
t_elapsed = perf_counter() - t_start

print(f"Time to read CZI file: {t_elapsed}.")

Time to read CZI file: 185.0351642947644.


In [7]:
# Get the shape of the image.
h, w = czi_img.shape[2:4]

In [8]:
# Calculate what the tile size and stride would be at scan magnification.
sf = SCAN_MAG / TILE_MAG  # tile mag -> scan mag
scan_tile_size = int(TILE_SIZE * sf)
scan_stride_size = int(STRIDE * sf)

# Get the xy coordinates of the tiles at scan magnification.
xys = [(x, y) for x in range(0, w, scan_stride_size) for y in range(0, h, scan_stride_size)]

# Create the location to save images.
save_dir = Path(SAVED_DIR)
img_dir = save_dir / "images"

img_dir.mkdir(parents=True, exist_ok=True)

In [9]:
def process_tile(x, y):
    """"""
    # Get the tile image.
    tile = czi_img[0, 0, y:y+scan_tile_size, x:x+scan_tile_size]
    
    # If the image is not scan_tile_size by scan_tile_size, pad the image.
    tile_h, tile_w = tile.shape[:2]
    if (tile_h, tile_w) != (scan_tile_size, scan_tile_size):
        # Pad with opencv.
        tile = cv.copyMakeBorder(
            tile, 
            0, 
            scan_tile_size - tile_h, 
            0, 
            scan_tile_size - tile_w, 
            cv.BORDER_CONSTANT, 
            value=RGB_FILL
        )
        
    # Resize the image now to be the tile size of interest.
    if tile.shape[:2] != (TILE_SIZE, TILE_SIZE):
        # Rescale the image to match the desired resolution.
        tile = cv.resize(tile, (TILE_SIZE, TILE_SIZE))
        
    x1, y1 = int(x / sf), int(y / sf)
    x2, y2 = x1 + TILE_SIZE, y1 + TILE_SIZE
        
    # Save the image.
    tile = cv.cvtColor(tile, cv.COLOR_RGB2BGR)  # opencv expects BGR image
    
    fp = img_dir / f"x1-{x1}_y1-{y1}_x2-{x2}_y2-{y2}.png"
    fp = str(fp.absolute())
    cv.imwrite(fp, tile)
    
    return fp, x1, y1, x2, y2
    
    
with Pool(NPROC) as pool:
    jobs = [
        pool.apply_async(
            func=process_tile,
            args=(xy[0], xy[1],)
        )
        for xy in xys
    ]
    
    tile_df = []
    
    for job in tqdm(jobs, desc="Processing tiles..."):
        tile_df.append(job.get())

tile_df = pd.DataFrame(data=tile_df, columns=['fp', 'x1', 'y1', 'x2', 'y2'])
tile_df.head()

Processing tiles...:   0%|          | 0/11040 [00:00<?, ?it/s]

,fp,x1,y1,x2,y2
0,/workspace/data/tiles/images/x1-0_y1-0_x2-1280...,0,0,1280,1280
1,/workspace/data/tiles/images/x1-0_y1-959_x2-12...,0,959,1280,2239
2,/workspace/data/tiles/images/x1-0_y1-1919_x2-1...,0,1919,1280,3199
3,/workspace/data/tiles/images/x1-0_y1-2879_x2-1...,0,2879,1280,4159
4,/workspace/data/tiles/images/x1-0_y1-3839_x2-1...,0,3839,1280,5119


In [13]:
# Predict 
print(Fore.CYAN + 'predicting tile labels...' + Style.RESET_ALL)
predict(
    str(img_dir), 
    str(SAVED_DIR), 
    "/workspace/code/best.pt", 
    device=None, 
    conf_thr=0.5, 
    iou_thr=0.8, 
    im_size=TILE_SIZE
)

predicting tile labels...


detect: weights=['/workspace/code/best.pt'], source=/workspace/data/tiles/images, data=data/coco128.yaml, imgsz=[1280, 1280], conf_thres=0.5, iou_thres=0.8, max_det=1000, device=0,1, view_img=False, save_txt=True, save_conf=True, save_crop=False, nosave=True, classes=None, agnostic_nms=True, augment=False, visualize=False, update=False, project=/workspace/data, name=tiles, exist_ok=True, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 2023-8-3 Python-3.8.13 torch-1.13.0a0+d0d6b1f CUDA:0 (NVIDIA TITAN V, 12057MiB)
                                                        CUDA:1 (NVIDIA TITAN V, 12057MiB)

Fusing layers... 
Model summary: 276 layers, 35254692 parameters, 0 gradients, 48.9 GFLOPs
image 1/11040 /workspace/data/tiles/images/x1-0_y1-0_x2-1280_y2-1280.png: 1280x1280 (no detections), 24.0ms
image 2/11040 /workspace/data/tiles/images/x1-0_y1-10557_x2-1280_y2-11837.png: 1280x1280 (no detections), 23.9ms
image 3/11040 /workspace/da

image 67/11040 /workspace/data/tiles/images/x1-0_y1-6718_x2-1280_y2-7998.png: 1280x1280 (no detections), 22.5ms
image 68/11040 /workspace/data/tiles/images/x1-0_y1-68145_x2-1280_y2-69425.png: 1280x1280 (no detections), 22.4ms
image 69/11040 /workspace/data/tiles/images/x1-0_y1-69104_x2-1280_y2-70384.png: 1280x1280 (no detections), 22.3ms
image 70/11040 /workspace/data/tiles/images/x1-0_y1-70064_x2-1280_y2-71344.png: 1280x1280 (no detections), 22.5ms
image 71/11040 /workspace/data/tiles/images/x1-0_y1-71024_x2-1280_y2-72304.png: 1280x1280 (no detections), 22.3ms
image 72/11040 /workspace/data/tiles/images/x1-0_y1-71984_x2-1280_y2-73264.png: 1280x1280 (no detections), 22.4ms
image 73/11040 /workspace/data/tiles/images/x1-0_y1-72943_x2-1280_y2-74223.png: 1280x1280 (no detections), 22.4ms
image 74/11040 /workspace/data/tiles/images/x1-0_y1-73903_x2-1280_y2-75183.png: 1280x1280 (no detections), 22.3ms
image 75/11040 /workspace/data/tiles/images/x1-0_y1-74863_x2-1280_y2-76143.png: 1280x1280 

image 138/11040 /workspace/data/tiles/images/x1-100777_y1-48949_x2-102057_y2-50229.png: 1280x1280 1 iNFT, 22.3ms
image 139/11040 /workspace/data/tiles/images/x1-100777_y1-49909_x2-102057_y2-51189.png: 1280x1280 (no detections), 22.4ms
image 140/11040 /workspace/data/tiles/images/x1-100777_y1-50868_x2-102057_y2-52148.png: 1280x1280 (no detections), 22.5ms
image 141/11040 /workspace/data/tiles/images/x1-100777_y1-51828_x2-102057_y2-53108.png: 1280x1280 (no detections), 22.4ms
image 142/11040 /workspace/data/tiles/images/x1-100777_y1-52788_x2-102057_y2-54068.png: 1280x1280 (no detections), 22.4ms
image 143/11040 /workspace/data/tiles/images/x1-100777_y1-53748_x2-102057_y2-55028.png: 1280x1280 (no detections), 22.4ms
image 144/11040 /workspace/data/tiles/images/x1-100777_y1-54707_x2-102057_y2-55987.png: 1280x1280 (no detections), 22.3ms
image 145/11040 /workspace/data/tiles/images/x1-100777_y1-55667_x2-102057_y2-56947.png: 1280x1280 (no detections), 22.3ms
image 146/11040 /workspace/data/t

image 206/11040 /workspace/data/tiles/images/x1-101737_y1-28793_x2-103017_y2-30073.png: 1280x1280 2 iNFTs, 22.3ms
image 207/11040 /workspace/data/tiles/images/x1-101737_y1-2879_x2-103017_y2-4159.png: 1280x1280 (no detections), 22.3ms
image 208/11040 /workspace/data/tiles/images/x1-101737_y1-29753_x2-103017_y2-31033.png: 1280x1280 5 iNFTs, 22.4ms
image 209/11040 /workspace/data/tiles/images/x1-101737_y1-30713_x2-103017_y2-31993.png: 1280x1280 1 iNFT, 22.4ms
image 210/11040 /workspace/data/tiles/images/x1-101737_y1-31673_x2-103017_y2-32953.png: 1280x1280 (no detections), 22.4ms
image 211/11040 /workspace/data/tiles/images/x1-101737_y1-32632_x2-103017_y2-33912.png: 1280x1280 (no detections), 22.3ms
image 212/11040 /workspace/data/tiles/images/x1-101737_y1-33592_x2-103017_y2-34872.png: 1280x1280 (no detections), 22.3ms
image 213/11040 /workspace/data/tiles/images/x1-101737_y1-34552_x2-103017_y2-35832.png: 1280x1280 (no detections), 22.4ms
image 214/11040 /workspace/data/tiles/images/x1-101

...

image 10716/11040 /workspace/data/tiles/images/x1-96938_y1-47989_x2-98218_y2-49269.png: 1280x1280 2 iNFTs, 22.3ms
image 10717/11040 /workspace/data/tiles/images/x1-96938_y1-4798_x2-98218_y2-6078.png: 1280x1280 (no detections), 22.3ms
image 10718/11040 /workspace/data/tiles/images/x1-96938_y1-48949_x2-98218_y2-50229.png: 1280x1280 (no detections), 22.3ms
image 10719/11040 /workspace/data/tiles/images/x1-96938_y1-49909_x2-98218_y2-51189.png: 1280x1280 (no detections), 22.4ms
image 10720/11040 /workspace/data/tiles/images/x1-96938_y1-50868_x2-98218_y2-52148.png: 1280x1280 2 iNFTs, 22.3ms
image 10721/11040 /workspace/data/tiles/images/x1-96938_y1-51828_x2-98218_y2-53108.png: 1280x1280 3 iNFTs, 22.3ms
image 10722/11040 /workspace/data/tiles/images/x1-96938_y1-52788_x2-98218_y2-54068.png: 1280x1280 3 iNFTs, 22.3ms
image 10723/11040 /workspace/data/tiles/images/x1-96938_y1-53748_x2-98218_y2-55028.png: 1280x1280 2 iNFTs, 22.3ms
image 10724/11040 /workspace/data/tiles/images/x1-96938_y1-54707_x

image 10784/11040 /workspace/data/tiles/images/x1-97898_y1-26874_x2-99178_y2-28154.png: 1280x1280 1 iNFT, 22.4ms
image 10785/11040 /workspace/data/tiles/images/x1-97898_y1-27833_x2-99178_y2-29113.png: 1280x1280 (no detections), 22.3ms
image 10786/11040 /workspace/data/tiles/images/x1-97898_y1-28793_x2-99178_y2-30073.png: 1280x1280 1 iNFT, 22.3ms
image 10787/11040 /workspace/data/tiles/images/x1-97898_y1-2879_x2-99178_y2-4159.png: 1280x1280 (no detections), 22.3ms
image 10788/11040 /workspace/data/tiles/images/x1-97898_y1-29753_x2-99178_y2-31033.png: 1280x1280 1 iNFT, 22.4ms
image 10789/11040 /workspace/data/tiles/images/x1-97898_y1-30713_x2-99178_y2-31993.png: 1280x1280 1 iNFT, 22.4ms
image 10790/11040 /workspace/data/tiles/images/x1-97898_y1-31673_x2-99178_y2-32953.png: 1280x1280 1 iNFT, 22.3ms
image 10791/11040 /workspace/data/tiles/images/x1-97898_y1-32632_x2-99178_y2-33912.png: 1280x1280 4 iNFTs, 22.4ms
image 10792/11040 /workspace/data/tiles/images/x1-97898_y1-33592_x2-99178_y2-34

image 10853/11040 /workspace/data/tiles/images/x1-97898_y1-8638_x2-99178_y2-9918.png: 1280x1280 (no detections), 22.3ms
image 10854/11040 /workspace/data/tiles/images/x1-97898_y1-87340_x2-99178_y2-88620.png: 1280x1280 (no detections), 22.3ms
image 10855/11040 /workspace/data/tiles/images/x1-97898_y1-9597_x2-99178_y2-10877.png: 1280x1280 (no detections), 22.3ms
image 10856/11040 /workspace/data/tiles/images/x1-97898_y1-959_x2-99178_y2-2239.png: 1280x1280 (no detections), 22.3ms
image 10857/11040 /workspace/data/tiles/images/x1-98858_y1-0_x2-100138_y2-1280.png: 1280x1280 (no detections), 22.3ms
image 10858/11040 /workspace/data/tiles/images/x1-98858_y1-10557_x2-100138_y2-11837.png: 1280x1280 (no detections), 22.4ms
image 10859/11040 /workspace/data/tiles/images/x1-98858_y1-11517_x2-100138_y2-12797.png: 1280x1280 (no detections), 22.3ms
image 10860/11040 /workspace/data/tiles/images/x1-98858_y1-12477_x2-100138_y2-13757.png: 1280x1280 (no detections), 22.4ms
image 10861/11040 /workspace/da

image 10924/11040 /workspace/data/tiles/images/x1-98858_y1-68145_x2-100138_y2-69425.png: 1280x1280 (no detections), 22.3ms
image 10925/11040 /workspace/data/tiles/images/x1-98858_y1-69104_x2-100138_y2-70384.png: 1280x1280 (no detections), 22.3ms
image 10926/11040 /workspace/data/tiles/images/x1-98858_y1-70064_x2-100138_y2-71344.png: 1280x1280 (no detections), 22.3ms
image 10927/11040 /workspace/data/tiles/images/x1-98858_y1-71024_x2-100138_y2-72304.png: 1280x1280 (no detections), 22.3ms
image 10928/11040 /workspace/data/tiles/images/x1-98858_y1-71984_x2-100138_y2-73264.png: 1280x1280 (no detections), 22.3ms
image 10929/11040 /workspace/data/tiles/images/x1-98858_y1-72943_x2-100138_y2-74223.png: 1280x1280 (no detections), 22.3ms
image 10930/11040 /workspace/data/tiles/images/x1-98858_y1-73903_x2-100138_y2-75183.png: 1280x1280 (no detections), 22.3ms
image 10931/11040 /workspace/data/tiles/images/x1-98858_y1-74863_x2-100138_y2-76143.png: 1280x1280 (no detections), 22.3ms
image 10932/1104

image 10993/11040 /workspace/data/tiles/images/x1-99818_y1-4798_x2-101098_y2-6078.png: 1280x1280 (no detections), 22.3ms
image 10994/11040 /workspace/data/tiles/images/x1-99818_y1-48949_x2-101098_y2-50229.png: 1280x1280 2 iNFTs, 22.3ms
image 10995/11040 /workspace/data/tiles/images/x1-99818_y1-49909_x2-101098_y2-51189.png: 1280x1280 3 iNFTs, 22.4ms
image 10996/11040 /workspace/data/tiles/images/x1-99818_y1-50868_x2-101098_y2-52148.png: 1280x1280 2 iNFTs, 22.4ms
image 10997/11040 /workspace/data/tiles/images/x1-99818_y1-51828_x2-101098_y2-53108.png: 1280x1280 (no detections), 22.3ms
image 10998/11040 /workspace/data/tiles/images/x1-99818_y1-52788_x2-101098_y2-54068.png: 1280x1280 (no detections), 22.3ms
image 10999/11040 /workspace/data/tiles/images/x1-99818_y1-53748_x2-101098_y2-55028.png: 1280x1280 (no detections), 22.3ms
image 11000/11040 /workspace/data/tiles/images/x1-99818_y1-54707_x2-101098_y2-55987.png: 1280x1280 (no detections), 22.4ms
image 11001/11040 /workspace/data/tiles/im

In [21]:
# Loop through tile images.
pred_df = []
    
for _, r in tqdm(tile_df.iterrows(), total=len(tile_df), desc="Reading tile predictions..."):
    # check for label text file
    label_fp = im_to_txt_path(r.fp)
    if isfile(label_fp):
        # read the label boxes, shift to be in WSI space (0,0 = top left of WSI)
        for pred in read_yolo_label(label_fp, im_shape=TILE_SIZE, shift=(-r.x1, -r.y1), convert=True):  
            label, x1, y1, x2, y2, conf = pred
            pred_df.append([r.fp, label, x1, y1, x2, y2, conf, Polygon([(x1, y1), (x2, y1), (x2, y2), (x1, y2)]), (x2-x1)*(y2-y1)])

pred_df = GeoDataFrame(
    pred_df, 
    columns=[
        'fp', 'label', 'x1', 'y1', 'x2', 'y2', 'conf', 'geometry', 
        'box_area'
    ]
)
print("Number of predictions found:", len(pred_df))
pred_df.head()

Reading tile predictions...:   0%|          | 0/11040 [00:00<?, ?it/s]

Number of predictions found: 3948


,fp,label,x1,y1,x2,y2,conf,geometry,box_area
0,/workspace/data/tiles/images/x1-2879_y1-34552_...,1.0,4019.000000,35186.999680,4083.000000,35254.999680,0.659927,"POLYGON ((4019.000 35187.000, 4083.000 35187.0...",4352.000000
1,/workspace/data/tiles/images/x1-3839_y1-33592_...,1.0,5009.999680,34211.000000,5117.999680,34261.000000,0.810965,"POLYGON ((5010.000 34211.000, 5118.000 34211.0...",5400.000000
2,/workspace/data/tiles/images/x1-3839_y1-34552_...,1.0,4019.000000,35186.999680,4083.000000,35254.999680,0.761305,"POLYGON ((4019.000 35187.000, 4083.000 35187.0...",4352.000000
3,/workspace/data/tiles/images/x1-4798_y1-29753_...,1.0,5513.999808,30393.000448,5582.999872,30466.000512,0.757994,"POLYGON ((5514.000 30393.000, 5583.000 30393.0...",5037.009088
4,/workspace/data/tiles/images/x1-4798_y1-33592_...,1.0,5008.999680,34210.999488,5124.999680,34259.999552,0.847481,"POLYGON ((5009.000 34210.999, 5125.000 34210.9...",5684.007424


In [25]:
pred_df = non_max_suppression(
        pred_df, NMS_IOU_THR
    ).sort_values(by='box_area', ascending=False).reset_index(drop=True)
pred_df = remove_contained_boxes(pred_df, CONTAINED_THR)

print("Number of predictions after cleanup:", len(pred_df))

Number of predictions after cleanup: 2321


In [26]:
"""Note that the boxes are in the coordinates of the tile magnification, in this example this is
40X."""

,fp,label,x1,y1,x2,y2,conf,geometry,box_area
0,/workspace/data/tiles/images/x1-74863_y1-46069...,1.0,74994.000320,46101.000640,75143.000000,46326.000320,0.661850,"POLYGON ((74994.000 46101.001, 75143.000 46101...",33524.880320
1,/workspace/data/tiles/images/x1-64305_y1-56627...,1.0,65048.000320,56631.000128,65232.000320,56779.999808,0.578030,"POLYGON ((65048.000 56631.000, 65232.000 56631...",27415.941120
2,/workspace/data/tiles/images/x1-48949_y1-16316...,1.0,49612.999360,16497.000960,49815.000000,16627.000320,0.662549,"POLYGON ((49612.999 16497.001, 49815.000 16497...",26259.953920
3,/workspace/data/tiles/images/x1-31673_y1-42230...,1.0,31797.999680,42705.999360,31941.999680,42872.999680,0.838681,"POLYGON ((31798.000 42705.999, 31942.000 42705...",24048.046080
4,/workspace/data/tiles/images/x1-96938_y1-35512...,1.0,97671.000128,36416.999680,97792.000192,36607.000320,0.760598,"POLYGON ((97671.000 36417.000, 97792.000 36417...",22990.089600
...,...,...,...,...,...,...,...,...,...
2379,/workspace/data/tiles/images/x1-84461_y1-27833...,1.0,85677.999680,28749.000512,85717.999680,28790.000448,0.664168,"POLYGON ((85678.000 28749.001, 85718.000 28749...",1639.997440
2380,/workspace/data/tiles/images/x1-65265_y1-59506...,1.0,65273.999936,60531.999680,65309.000000,60577.999680,0.565013,"POLYGON ((65274.000 60532.000, 65309.000 60532...",1610.002944
2381,/workspace/data/tiles/images/x1-56627_y1-7678_...,1.0,57588.000448,7683.000000,57627.000512,7721.000000,0.510947,"POLYGON ((57588.000 7683.000, 57627.001 7683.0...",1482.002432
2383,/workspace/data/tiles/images/x1-93099_y1-14396...,1.0,93100.000000,14657.000320,93129.000064,14701.000320,0.506141,"POLYGON ((93100.000 14657.000, 93129.000 14657...",1276.002816
